In [24]:
import sys
sys.path.append("../")

from src.augmentation import add_noise
from src.config import project_paths
from pathlib import Path
import pandas as pd
from copy import deepcopy
from typing import Dict, List
import os
from tqdm import tqdm

In [2]:
sample_annotations_path = project_paths.get_data_file_path(data_file="annotations_original.json")
sample_annotations: List[Dict[str, str]] = list(pd.read_json(sample_annotations_path, orient="records").to_dict(orient="index").values())
amplitude_dir = project_paths.get_data_file_path("amplitude_audio_files")
amplitude_dir.mkdir(exist_ok=True)

In [31]:
amplitude_annotations = []
for annotation in tqdm(sample_annotations, total=len(sample_annotations)):
    rel_audio_path = Path(f"../" + annotation["wav_path"])
    abs_audio_path = Path(f"../data/generated/amplitude_audio_files") / Path(*rel_audio_path.parts[-3:])
    os.makedirs(os.path.dirname(abs_audio_path), exist_ok=True)
    add_noise.add_more_gaussian_noise(audio_file_path=rel_audio_path, target_file_path=abs_audio_path)
    subsampling_annotation = deepcopy(annotation)
    subsampling_annotation["wav_path"] = Path(*abs_audio_path.parts[1:])
    subsampling_annotation["original_wav_path"] = rel_audio_path
    amplitude_annotations.append(subsampling_annotation)

100%|██████████| 1341/1341 [00:25<00:00, 51.73it/s]


In [32]:
len(amplitude_annotations)

1341

In [33]:
pd.DataFrame(amplitude_annotations).to_json(project_paths.get_data_file_path("annotations_multiply_amplitude.json"),orient="records",default_handler=str)